In [1]:
import main
import plot_inequality as plot_ineq
import time
import importlib
import io_manager
import numpy as np
importlib.reload(plot_ineq)
#run stuff here
start_time = time.perf_counter()
print("Running main")

of_name = None
main_path = "C:/Users/Ben/My Drive/PhD/PhD Year 3/3rd Year Paper/Model/My Code/MH_Model/my_code/model_uncert/"
input_path = main_path + "input/50p_age_moms/"
trans_path_uncond = input_path + "MH_trans_uncond_age.csv"

trans_path_50p = input_path + "MH_trans_by_MH_clust_age.csv"
type_path_50p = input_path + "MH_clust_50p_age_pop_shares.csv"

do_dpi_calib = False
output_flag = False
myPars, myShocks, sols, sims = main.main_io(main_path, out_folder_name = of_name, 
                                            H_trans_uncond_path = trans_path_uncond, 
                                            H_trans_path = trans_path_50p, H_type_pop_share_path = type_path_50p,
                                            output_flag = output_flag, do_dpi_calib = do_dpi_calib)

Running main
*****Running main_io with default out_folder_name*****
Solver ran in 7.61768760002451 seconds
Calibrating with alpha_lab_targ = 0.33497447, w0_mean_targ = 2.1988928, w0_sd_targ = 0.29847395, 
                                        w1_targ = 0.2928040000000003, w2_targ = 0.2746706000000003, wH_targ = 0.051316846, phi_H_targ = 0.053118114,
                                        dpi_BB_targ = 0.50886095, dpi_GG_targ = 0.34358001, eps_gg_targ = 0.39124098057692314
***** Calibration iteration 0 *****
Calibrating epsilon_gg
my_eps_gg_mom: 0.39188756703943345
Calibrating w0_mu
Calibrating w0_sigma
***** Calibration iteration 1 *****
Calibrating w0_mu
Calibrating w0_sigma
***** Calibration iteration 2 *****
Calibrating w0_mu
Calibrating w0_sigma
Calibrating w1
Calibrating w2
***** Calibration iteration 3 *****
Calibrating w0_mu
Calibrating w0_sigma
Calibrating w1
Calibrating w2
***** Calibration iteration 4 *****
Calibrating w0_mu
Calibrating w0_sigma
Calibrating w1
Calibrating 

In [34]:
from pars_shocks import Pars, Shocks
import pars_shocks
importlib.reload(main)

# calib phi_H: time cost counterfactual
# myPars_tc, myShocks_tc, sols_tc, sims_tc = main.main_io(main_path, out_folder_name = of_name,
#                                                         H_trans_uncond_path = trans_path_uncond, 
#                                                         H_trans_path = trans_path_50p, H_type_pop_share_path = type_path_50p,
#                                                         output_flag = False, do_dpi_calib = False, do_phi_H_calib=True) 
# H = good counterfactual
# myPars_H_good: Pars = main.pars_factory(main_path, 
#                                         H_trans_uncond_path = trans_path_uncond, 
#                                         H_trans_path = trans_path_50p, H_type_pop_share_path = type_path_50p)
myPars_H_good = myPars
myShocks_H_good: Shocks = Shocks(myPars_H_good)

H_hist = np.ones(myPars.state_space_shape_sims, dtype=np.int64)
myShocks_H_good.H_hist = H_hist
print("Pre - np.mean(H_hist):", np.mean(H_hist))

myPars_H_good, myShocks_H_good, sols_H_good, sims_H_good = main.main_io(main_path, out_folder_name = of_name, 
                                                                    H_trans_uncond_path = trans_path_uncond, 
                                                                    H_trans_path = trans_path_50p, H_type_pop_share_path = type_path_50p,
                                                                    myPars = myPars_H_good, myShocks=myShocks_H_good, 
                                                                    calib_flag=False, sim_no_calib=True, output_flag = False, 
                                                                    do_wH_calib = False, do_phi_H_calib=False, 
                                                                    do_dpi_calib = False, do_eps_gg_calib=False)

H_hist = myShocks_H_good.H_hist
print("Post - np.mean(H_hist):", np.mean(H_hist))
# # wH = 0 counterfactual
# myPars_no_wH: Pars = main.pars_factory(main_path,
#                                         H_trans_uncond_path = trans_path_uncond, 
#                                         H_trans_path = trans_path_50p, H_type_pop_share_path = type_path_50p)
# myPars_no_wH.wH_coeff = 0.0
# myPars_no_wH, myShocks_no_wH, sols_no_wH, sims_no_wH = main.main_io(main_path, out_folder_name = of_name,
#                                                                     H_trans_uncond_path = trans_path_uncond, 
#                                                                     H_trans_path = trans_path_50p, H_type_pop_share_path = type_path_50p,
#                                                                     myPars = myPars_no_wH, myShocks=myShocks, 
#                                                                     output_flag = False, do_wH_calib = False, do_dpi_calib = False)

Pre - np.mean(H_hist): 1.0
*****Running main_io with default out_folder_name*****
Solver ran in 0.5891511999943759 seconds
Simulate ran in 0.7251515999960247 seconds
Post - np.mean(H_hist): 1.0


In [35]:
import model_uncert as model
from typing import Tuple
age_25_ind = np.where(myPars.age_grid == 25)[0][0] 
age_55_ind = np.where(myPars.age_grid == 55)[0][0]

lab_earn_trim: np.array = sims["lab_earnings"][:, :, :, age_25_ind:age_55_ind+1]*12
# lab_earn_trim_no_wH: np.array = sims_no_wH["lab_earnings"][:, :, :, age_25_ind:age_55_ind+1]*12
lab_earn_trim_H_good: np.array = sims_H_good["lab_earnings"][:, :, :, age_25_ind:age_55_ind+1]*12
# lab_earn_trim_tc: np.array = sims_tc["lab_earnings"][:, :, :, age_25_ind:age_55_ind+1]*12

log_lab_earn = np.log(lab_earn_trim)
# log_lab_earn_no_wH = np.log(lab_earn_trim_no_wH)
log_lab_earn_H_good = np.log(lab_earn_trim_H_good)
# log_lab_earn_tc = np.log(lab_earn_trim_tc)

In [36]:


def calc_wstats(earnings: np.array, weights_lab_fe: np.array, weight_H_type: np.array) -> dict:

    weights = np.outer(weights_lab_fe, weight_H_type)
    sim_num = earnings.shape[2]
    age_num = earnings.shape[3]
    weights_exp = np.repeat(weights[:, :, np.newaxis], sim_num, axis=2)
    weights_exp = np.repeat(weights_exp[:, :, :, np.newaxis], age_num, axis=3)

    flat_earns = earnings.flatten()
    flat_weights = weights_exp.flatten()

    mean_earn = np.average(flat_earns, weights=flat_weights)

    #calculate the weighted median
    sorted_inds = np.argsort(flat_earns)
    sorted_earns = flat_earns[sorted_inds]
    sorted_weights = flat_weights[sorted_inds]
    cum_weights = np.cumsum(sorted_weights)
    cutoff = 0.5 * cum_weights[-1]
    median_earn = sorted_earns[np.searchsorted(cum_weights, cutoff)]

    #calc weighted variance
    var_earn = np.average((flat_earns - mean_earn)**2, weights=flat_weights)
    #calc weighted std dev
    std_earn = np.sqrt(var_earn)

    #calc weighted min and max
    min_earn = np.min(flat_earns)
    max_earn = np.max(flat_earns)

    # Calculate weighted percentiles
    pct_10 = np.percentile(sorted_earns, 10)
    pct_25 = np.percentile(sorted_earns, 25)
    pct_75 = np.percentile(sorted_earns, 75)
    pct_90 = np.percentile(sorted_earns, 90)

    # Step 5: Return the results in a dictionary
    ret_dict = {
        "mean": mean_earn,
        "median": median_earn,
        "var": var_earn,
        "std_dev": std_earn,
        "min": min_earn,
        "max": max_earn,
        "P10": pct_10,
        "P25": pct_25,
        "P75": pct_75,
        "P90": pct_90
    }

    return ret_dict


In [37]:

import numpy as np

def calculate_weighted_lt_stats(earnings: np.array, weights_lab_fe: np.array, weights_H_type: np.array) -> dict:
    # Step 1: Calculate lifetime earnings (annualized mean earnings from ages 25-55)
    # Average across the last axis (age), assuming the earnings matrix is already restricted to ages 25-55.
    lifetime_earnings = earnings.mean(axis=-1)  # Shape: (lab_fe, H_type, simulation_number)
    
    # Step 2: Expand the weights to match the dimensions of lifetime_earnings
    # Create a grid of weights by multiplying the lab_fe and H_type weights
    weights = np.outer(weights_lab_fe, weights_H_type)  # Shape: (lab_fe, H_type)
    
    # Reshape the weights to broadcast over the (simulation_number) dimension
    sim_num = lifetime_earnings.shape[2]
    weights_expanded = np.repeat(weights[:, :, np.newaxis], sim_num, axis=2)  # Shape: (lab_fe, H_type, simulation_number)

    # Step 3: Flatten the earnings and weights for weighted calculation
    flattened_earnings = lifetime_earnings.flatten()  # Shape: (lab_fe * H_type * simulation_number,)
    flattened_weights = weights_expanded.flatten()    # Shape: (lab_fe * H_type * simulation_number,)

    # Step 4: Compute weighted statistics

    # Calculate weighted mean
    mean_earn = np.average(flattened_earnings, weights=flattened_weights)

    # Calculate weighted median
    sorted_indices = np.argsort(flattened_earnings)
    sorted_earnings = flattened_earnings[sorted_indices]
    sorted_weights = flattened_weights[sorted_indices]
    cumsum_weights = np.cumsum(sorted_weights)
    cutoff = 0.5 * cumsum_weights[-1]
    median_earn = sorted_earnings[np.searchsorted(cumsum_weights, cutoff)]

    # Calculate weighted variance
    var_earn = np.average((flattened_earnings - mean_earn) ** 2, weights=flattened_weights)

    # Calculate weighted standard deviation
    std_earn = np.sqrt(var_earn)

    # Calculate weighted min and max
    min_earn = np.min(flattened_earnings)
    max_earn = np.max(flattened_earnings)

    # Calculate weighted percentiles
    pct_10 = np.percentile(sorted_earnings, 10)
    pct_25 = np.percentile(sorted_earnings, 25)
    pct_75 = np.percentile(sorted_earnings, 75)
    pct_90 = np.percentile(sorted_earnings, 90)

    # Step 5: Return the results in a dictionary
    ret_dict = {
        "mean": mean_earn,
        "median": median_earn,
        "var": var_earn,
        "std_dev": std_earn,
        "min": min_earn,
        "max": max_earn,
        "P10": pct_10,
        "P25": pct_25,
        "P75": pct_75,
        "P90": pct_90
    }
    
    return ret_dict





In [38]:
earnings = lab_earn_trim 
weights_lab_fe = myPars.lab_fe_weights
weights_H_type = myPars.H_type_perm_weights
lt_stats = calculate_weighted_lt_stats(earnings, weights_lab_fe, weights_H_type)
for key in lt_stats:
    print(key, lt_stats[key])

print()

# earnings_tc = lab_earn_trim_tc
# weights_lab_fe = myPars_tc.lab_fe_weights
# weights_H_type = myPars_tc.H_type_perm_weights
# lt_stats_tc = calculate_weighted_lt_stats(earnings_tc, weights_lab_fe, weights_H_type)
# for key in lt_stats_tc:
    # print(key, lt_stats_tc[key])

# print()

# earnings_no_wH = lab_earn_trim_no_wH 
# weights_lab_fe = myPars_no_wH.lab_fe_weights
# weights_H_type = myPars_no_wH.H_type_perm_weights
# lt_stats_no_wH = calculate_weighted_lt_stats(earnings_no_wH, weights_lab_fe, weights_H_type)
# for key in lt_stats_no_wH:
#     print(key, lt_stats_no_wH[key])

# print()

earnings_H_good = lab_earn_trim_H_good
weights_lab_fe = myPars_H_good.lab_fe_weights
weights_H_type = myPars_H_good.H_type_perm_weights
lt_stats_H_good = calculate_weighted_lt_stats(earnings_H_good, weights_lab_fe, weights_H_type)
for key in lt_stats_H_good:
    print(key, lt_stats_H_good[key])



mean 19394.256909076194
median 18094.55309261539
var 35527267.98239053
std_dev 5960.475482911622
min 10150.528096556149
max 43856.00777400896
P10 13237.80551742606
P25 17833.567418027465
P75 35615.96372111855
P90 40228.34132994098

mean 20254.699575392362
median 18650.619949076983
var 38590257.15718579
std_dev 6212.10569430252
min 10964.002145866283
max 44761.48686697786
P10 14275.839709168336
P25 18556.299132356977
P75 36735.31465466437
P90 41313.82747321312


In [39]:
var_earn = lt_stats["var"]
var_earn_H = lt_stats_H_good["var"]
print(f"var_earn: {round(var_earn/1000, 2)}")
print(f"var_earn_H: {round(var_earn_H/1000, 2)}")
print(f"liftetime earn var %: {(var_earn - var_earn_H)/var_earn*100}")
print(f"log lifetime earn var %: {(np.log(var_earn) - np.log(var_earn_H))*100}")

var_earn: 35527.27
var_earn_H: 38590.26
liftetime earn var %: -8.621516228924401
log lifetime earn var %: -8.269932554276593


In [40]:
import pandas as pd
import my_toolbox as tb
outpath = myPars.path + "output/"

lt_stats_data_path = input_path + "lt_earn_stats.csv" 
lt_stats_df =  pd.read_csv(lt_stats_data_path)
# change the column names
lt_stats_df.columns = ["mean", "std_dev",  "P10", "P25", "P50", "P75", "P90"]
# convert to dictionary
lt_stats_data = lt_stats_df.to_dict(orient = "records")[0]
for key in lt_stats_df:
    print(key, lt_stats_df[key])

r2f =lambda x : round(x/1000, 2) 
r2 = lambda x : round(x, 2)
rf = lambda x : round(x/1000)

tab = [
    "\\documentclass[border=3mm,preview]{standalone}",
    "\\usepackage{booktabs} \n",
    "\\usepackage{caption} \n",
    "\\usepackage{pdflscape} \n",
    "\\begin{document}\n",
    "\\begin{landscape}\n",
    # "\\textit{Liftime Earnings Statistics} \\\\ \n",
    "\\small\n",
    "\\begin{table} \n",
    "\\center\\caption*{Annualized Lifetime Earnings Statistics (1000s GBP)} \n",
    "\\begin{tabular}{l | l l l | l l l l l | l l l} \n",
    "\\toprule \n",
    " & \\multicolumn{3}{c}{Summary} & \\multicolumn{5}{c}{Percentiles} & \\multicolumn{3}{c}{Ratios} \\\\ \n",
    # "\\hline \n",
    "Source & Mean & Var. & SD & 10th & 25th & 50th & 75th & 90th & 90/10 & 90/50 & 50/10 \\\\ \n", 
    "\\midrule \n",
    f"""Baseline Model & {r2f(lt_stats['mean'])} & {rf(lt_stats['var'])} & {r2f(lt_stats['std_dev'])} & 
        {r2f(lt_stats['P10'])} & {r2f(lt_stats['P25'])} & {r2f(lt_stats['median'])} & {r2f(lt_stats['P75'])} & {r2f(lt_stats['P90'])} &
        {r2(lt_stats['P90']/lt_stats['P10'])} & {r2(lt_stats['P90']/lt_stats['median'])} & {r2(lt_stats['median']/lt_stats['P10'])} \\\\ \n""",
    f"""$H = Good$ & {r2f(lt_stats_H_good['mean'])} & {rf(lt_stats_H_good['var'])} & {r2f(lt_stats_H_good['std_dev'])} & 
        {r2f(lt_stats_H_good['P10'])} & {r2f(lt_stats_H_good['P25'])} & {r2f(lt_stats_H_good['median'])} & {r2f(lt_stats_H_good['P75'])} & {r2f(lt_stats_H_good['P90'])} &
        {r2(lt_stats_H_good['P90']/lt_stats_H_good['P10'])} & {r2(lt_stats_H_good['P90']/lt_stats_H_good['median'])} 
        & {r2(lt_stats_H_good['median']/lt_stats_H_good['P10'])} \\\\ \n""",
    # f"""$Time Cost$ & {r2f(lt_stats_tc['mean'])} & {rf(lt_stats_tc['var'])} & {r2f(lt_stats_tc['std_dev'])} & 
    #     {r2f(lt_stats_tc['P10'])} & {r2f(lt_stats_tc['P25'])} & {r2f(lt_stats_tc['median'])} & {r2f(lt_stats_tc['P75'])} & {r2f(lt_stats_tc['P90'])} &
    #     {r2(lt_stats_tc['P90']/lt_stats_tc['P10'])} & {r2(lt_stats_tc['P90']/lt_stats_tc['median'])} 
    #     & {r2(lt_stats_tc['median']/lt_stats_tc['P10'])} \\\\ \n""",
    # f"""Data & {r2f(lt_stats_data['mean'])} & - & {r2f(lt_stats_data['std_dev'])} & 
    #     {r2f(lt_stats_data['P10'])} & {r2f(lt_stats_data['P25'])} & {r2f(lt_stats_data['P50'])} & {r2f(lt_stats_data['P75'])} & {r2f(lt_stats_data['P90'])} &
    #     {r2(lt_stats_data['P90']/lt_stats_data['P10'])} & {r2(lt_stats_data['P90']/lt_stats_data['P50'])} & {r2(lt_stats_data['P50']/lt_stats_data['P10'])} \\\\ \n""",
    "\\bottomrule \n",   
    "\\end{tabular}\n",
    "\\end{table}\n",
    "\\end{landscape}\n",
    "\\end{document}\n"
]

file_name = "lt_earnings_stats_tc_counter.tex"
tb.list_to_tex(outpath, file_name, tab)
tb.tex_to_pdf(outpath, file_name)


mean 0    20274.967
Name: mean, dtype: float64
std_dev 0    10876.716
Name: std_dev, dtype: float64
P10 0    8689.2451
Name: P10, dtype: float64
P25 0    12836.752
Name: P25, dtype: float64
P50 0    18606.549
Name: P50, dtype: float64
P75 0    26088.908
Name: P75, dtype: float64
P90 0    34577.691
Name: P90, dtype: float64
PDF successfully created at C:/Users/Ben/My Drive/PhD/PhD Year 3/3rd Year Paper/Model/My Code/MH_Model/my_code/model_uncert/output/lt_earnings_stats_tc_counter.pdf


In [41]:
earnings = log_lab_earn 
weights_lab_fe = myPars.lab_fe_weights
weights_H_type = myPars.H_type_perm_weights
lt_stats = calculate_weighted_lt_stats(earnings, weights_lab_fe, weights_H_type)
for key in lt_stats:
    print(key, lt_stats[key])

print()

# earnings_no_wH = log_lab_earn_no_wH
# weights_lab_fe = myPars_no_wH.lab_fe_weights
# weights_H_type = myPars_no_wH.H_type_perm_weights
# lt_stats_no_wH = calculate_weighted_lt_stats(earnings_no_wH, weights_lab_fe, weights_H_type)
# for key in lt_stats_no_wH:
#     print(key, lt_stats_no_wH[key])

# print()

earnings_H_good = log_lab_earn_H_good
weights_lab_fe = myPars_H_good.lab_fe_weights
weights_H_type = myPars_H_good.H_type_perm_weights
lt_stats_H_good = calculate_weighted_lt_stats(earnings_H_good, weights_lab_fe, weights_H_type)
for key in lt_stats_H_good:
    print(key, lt_stats_H_good[key])

mean 9.797021414886327
median 9.77373426946427
var 0.08906589851728607
std_dev 0.2984391035325064
min 9.19287746997032
max 10.661692356706128
P10 9.457423141181915
P25 9.756988361006
P75 10.448386388200692
P90 10.574838921139008

mean 9.847666403995976
median 9.81218398750826
var 0.08869969560616094
std_dev 0.2978249412090279
min 9.275398015630506
max 10.687652700568558
P10 9.536907912201414
P25 9.805693897104268
P75 10.48586088681173
P90 10.606746427676413


In [42]:
var_earn = lt_stats["var"]
var_earn_H = lt_stats_H_good["var"]
print(f"var_earn: {var_earn}")
print(f"var_earn_H: {var_earn_H}")
print(f"liftetime earn var %: {((var_earn - var_earn_H)/var_earn_H)*100}")
print(f"log lifetime earn var %: {(np.log(var_earn) - np.log(var_earn_H))*100}")
print(f'raw diff: {var_earn - var_earn_H}')

var_earn: 0.08906589851728607
var_earn_H: 0.08869969560616094
liftetime earn var %: 0.41285700996215574
log lifetime earn var %: 0.41200709389772605
raw diff: 0.0003662029111251297
